# Heatmap of NYC Subway Station Traffic

### Import Packages

In [19]:
import pandas as pd
from datetime import datetime
import pickle
import folium

### Load Turnstile Data

In [2]:
# 2017 Turnstile data from "https://data.ny.gov/Transportation/Turnstile-Usage-Data-2017/v5y5-mwpb"
# Replace the path below with the file on your local machine
path = "/Users/kc/OneDrive/Projects/projectbenson/data/data_2017.csv"

df = pd.read_csv(path, sep=',')

/Users/kc/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3049: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
df.head()

,C/A,Unit,SCP,Station,Line Name,Division,Date,Time,Description,Entries,Exits
0,A002,R051,02-00-00,59 ST,NQR456W,BMT,12/22/2017,23:00:00,REGULAR,6455787,2184977
1,A002,R051,02-00-00,59 ST,NQR456W,BMT,12/22/2017,19:00:00,REGULAR,6455505,2184926
2,A002,R051,02-00-00,59 ST,NQR456W,BMT,12/22/2017,15:00:00,REGULAR,6454819,2184867
3,A002,R051,02-00-00,59 ST,NQR456W,BMT,12/22/2017,11:00:00,REGULAR,6454437,2184773
4,A002,R051,02-00-00,59 ST,NQR456W,BMT,12/22/2017,07:00:00,REGULAR,6454285,2184501


In [4]:
df.columns

Index(['C/A', 'Unit', 'SCP', 'Station', 'Line Name', 'Division', 'Date',
       'Time', 'Description', 'Entries',
       'Exits                                                     '],
      dtype='object')

In [5]:
df.dtypes

C/A                                                           object
Unit                                                          object
SCP                                                           object
Station                                                       object
Line Name                                                     object
Division                                                      object
Date                                                          object
Time                                                          object
Description                                                   object
Entries                                                        int64
Exits                                                          int64
dtype: object

### Clean & Transform Turnstile Data

In [6]:
# Remove whitespace from column names
df.columns = df.columns.str.strip()

# Drop irrelevant columns
df.drop(columns=["Line Name", "Division", "Description"], axis=1, inplace=True)

In [7]:
# This cell will take some time to run
# Concatenate Date & Time into DateTime column DT
df["DT"] = df[['Date', 'Time']].apply(lambda x: ' '.join(x), axis=1)

# Convert DT into DateTime format
df["DT"] = pd.to_datetime(df["DT"], infer_datetime_format=True)

# Concatenate C/A, Unit & SCP into 1 Unique Identifier Column
df["ID"] = df[["C/A", "Unit", "SCP"]].apply(lambda x: ' '.join(x), axis=1)

# Rearrange Columns
df = df[["ID", "DT", "Station", "Entries", "Exits"]]

In [8]:
# This cell will take some time to run
# Sort DT in descending order
df.sort_values(by=['ID', 'DT'], ascending=[True, False], inplace=True)

# Apply pd.Series.diff() on ["Entries"] and ["Exits"] within each ["ID"] group to obtain value of incremental change
df['Entry_Diff'] = df.groupby(['ID'])['Entries'].apply(lambda x: abs(x.diff()))
df['Exit_Diff'] = df.groupby(['ID'])['Exits'].apply(lambda x: abs(x.diff()))

# Sum values of incremental change for ["Entries"] and ["Exits"] for overall Traffic within time interval
df['Traffic'] = df["Entry_Diff"] + df["Exit_Diff"]

In [9]:
# Remove outliers by taking only values below the 99th percentile
f1 = df["Entry_Diff"].quantile(0.99)
df = df[df["Entry_Diff"] < f1]

f2 = df["Exit_Diff"].quantile(0.99)
df = df[df["Exit_Diff"] < f2]

f3 = df["Traffic"].quantile(0.99)
df = df[df["Traffic"] < f3]

### Pickling Data for Faster Loading

In [2]:
# Pickle df to load data faster next time

# To Save Pickle
# with open('df.pickle', 'wb') as to_write:
#     pickle.dump(df, to_write)
    
# To Load Pickle
# with open('df.pickle','rb') as read_file:
#     df = pickle.load(read_file)

# Check
df.head()

,ID,DT,Station,Entries,Exits,Entry_Diff,Exit_Diff,Traffic
1,A002 R051 02-00-00,2017-12-22 19:00:00,59 ST,6455505,2184926,282.0,51.0,333.0
2,A002 R051 02-00-00,2017-12-22 15:00:00,59 ST,6454819,2184867,686.0,59.0,745.0
3,A002 R051 02-00-00,2017-12-22 11:00:00,59 ST,6454437,2184773,382.0,94.0,476.0
4,A002 R051 02-00-00,2017-12-22 07:00:00,59 ST,6454285,2184501,152.0,272.0,424.0
5,A002 R051 02-00-00,2017-12-22 03:00:00,59 ST,6454273,2184470,12.0,31.0,43.0


### Load Coordinate Data

In [20]:
# Bringing in station coordinate data 
# Station coordinate data from "https://data.cityofnewyork.us/Transportation/Subway-Stations/arq3-7z49"
# Replace the path below with the file on your local machine
coords_df = pd.read_csv("/Users/kc/OneDrive/Projects/projectbenson/data/station_coordinates.csv", sep=',')

coords_df.head()

,URL,OBJECTID,NAME,the_geom,LINE,NOTES
0,http://web.mta.info/nyct/service/,1,Astor Pl,POINT (-73.99106999861966 40.73005400028978),4-6-6 Express,"4 nights, 6-all times, 6 Express-weekdays AM s..."
1,http://web.mta.info/nyct/service/,2,Canal St,POINT (-74.00019299927328 40.71880300107709),4-6-6 Express,"4 nights, 6-all times, 6 Express-weekdays AM s..."
2,http://web.mta.info/nyct/service/,3,50th St,POINT (-73.98384899986625 40.76172799961419),1-2,"1-all times, 2-nights"
3,http://web.mta.info/nyct/service/,4,Bergen St,POINT (-73.97499915116808 40.68086213682956),2-3-4,"4-nights, 3-all other times, 2-all times"
4,http://web.mta.info/nyct/service/,5,Pennsylvania Ave,POINT (-73.89488591154061 40.66471445143568),3-4,"4-nights, 3-all other times"


### Cleaning & Transforming Coordinate Data

In [21]:
# Remove useless characters
coords_df["the_geom"] = coords_df["the_geom"].str.replace("POINT \(|\)", "")

# Split coordinates into lat & lon
coords = coords_df["the_geom"].str.split(expand=True)

# Reassign into new columns
coords_df["lon"] = coords[0].astype(float)
coords_df["lat"] = coords[1].astype(float)

# Take only relevant columns
coords_df = coords_df[["NAME", "lat", "lon"]]

In [22]:
coords_df.dtypes

NAME     object
lat     float64
lon     float64
dtype: object

In [23]:
# Check for duplicates
coords_df["NAME"].duplicated().sum()

118

In [24]:
# Drop duplicates
coords_df["NAME"].drop_duplicates(inplace=True)

In [25]:
# Recheck for duplicates
coords_df["NAME"].duplicated().sum()

0

In [26]:
# Map Station Names from Coordinate dataset to Turnstile Data set for joining
# Only Top 10 stations explicitly mapped; others will be joined as much as possible
mapping = {"34th St - Penn Station" : "34 ST-PENN STA",
          "Times Sq - 42nd St" : "TIMES SQ-42 ST",
          "42nd St - Port Authority Bus Term" : "42 ST-PORT AUTH",
          "Herald Sq - 34th St" : "34 ST-HERALD SQ",
          "Grand Central - 42nd St" : "GRD CNTRL-42 ST",
          "Union Sq - 14th St" : "14 ST-UNION SQ",
          "Flushing - Main St" : "FLUSHING-MAIN",
          "59th St - Columbus Circle" : "59 ST COLUMBUS",
          "Atlantic Av - Barclay's Center" : "ATL AV-BARCLAY"}

coords_df.replace({"NAME" : mapping}, inplace=True)

# Making names uppercase
coords_df["NAME"] = coords_df["NAME"].str.upper()

# Remove th, rd and nd from station names with regex for better chance of joining
coords_df["NAME"] = coords_df["NAME"].str.replace("(?<=\d)TH|(?<=\d)RD|(?<=\d)ND\b", "")

In [27]:
# Check
coords_df.head()

,NAME,lat,lon
0,ASTOR PL,40.7300540002897833,-73.9910699986196647
1,CANAL ST,40.7188030010770916,-74.0001929992732812
2,50 ST,40.7617279996141875,-73.9838489998662538
3,BERGEN ST,40.6808621368295604,-73.9749991511680776
4,PENNSYLVANIA AVE,40.6647144514356782,-73.8948859115406123


### Left Joining Turnstile and Coordinate Datasets

In [28]:
# Joining Coordinate Data with Turnstile Data
full_df = df.merge(coords_df, how='left', left_on='Station', right_on='NAME')

# Check
full_df.head()

,ID,DT,Station,Entries,Exits,Entry_Diff,Exit_Diff,Traffic,NAME,lat,lon
0,A002 R051 02-00-00,2017-12-22 19:00:00,59 ST,6455505,2184926,282.0,51.0,333.0,59 ST,40.6413616662837995,-74.0178809995398694
1,A002 R051 02-00-00,2017-12-22 15:00:00,59 ST,6454819,2184867,686.0,59.0,745.0,59 ST,40.6413616662837995,-74.0178809995398694
2,A002 R051 02-00-00,2017-12-22 11:00:00,59 ST,6454437,2184773,382.0,94.0,476.0,59 ST,40.6413616662837995,-74.0178809995398694
3,A002 R051 02-00-00,2017-12-22 07:00:00,59 ST,6454285,2184501,152.0,272.0,424.0,59 ST,40.6413616662837995,-74.0178809995398694
4,A002 R051 02-00-00,2017-12-22 03:00:00,59 ST,6454273,2184470,12.0,31.0,43.0,59 ST,40.6413616662837995,-74.0178809995398694


### Subsetting Data for HeatMap

In [29]:
#  Monthly Data
monthly_data = full_df.groupby(["Station", "lat", "lon", full_df["DT"].dt.month])["Traffic"].mean().reset_index()

# Check
monthly_data.head()

,Station,lat,lon,DT,Traffic
0,103 ST,40.7994460003348252,-73.9683789996081771,1,476.6387874360847263
1,103 ST,40.7994460003348252,-73.9683789996081771,2,461.9665722379603494
2,103 ST,40.7994460003348252,-73.9683789996081771,3,496.9138943248532314
3,103 ST,40.7994460003348252,-73.9683789996081771,4,499.9649253731343492
4,103 ST,40.7994460003348252,-73.9683789996081771,5,492.1904090267983065


In [30]:
# Grab January's Data for all Stations
month_1 = monthly_data.loc[monthly_data["DT"] == 1]

# Check
month_1.head()

,Station,lat,lon,DT,Traffic
0,103 ST,40.7994460003348252,-73.9683789996081771,1,476.6387874360847263
12,104 ST,40.6817110010911946,-73.8376830006099709,1,96.9867947178871503
24,110 ST,40.7950200001131051,-73.9442499968716334,1,450.4428706326723386
36,111 ST,40.6843310000123779,-73.8321629984538816,1,224.4622705642420044
48,116 ST,40.8020979991330037,-73.9496250009690499,1,349.2568852059468441


### Plotting Map & HeatMap

In [31]:
# Set starting location for map to be the centre of all coordinates 
location = monthly_data['lat'].mean(), monthly_data['lon'].mean()
m = folium.Map(location=location, zoom_start=11.5, tiles="Stamen Toner")

# Transform data into list format
heat_data = month_1[['lat','lon','Traffic']].values.tolist()

In [32]:
# Plot HeatMap with parameters
from folium.plugins import HeatMap
HeatMap(heat_data, radius=10, gradient={0.2: 'blue', 0.5: 'orange', 0.6: 'red'}, min_opacity=0.7, max_opacity=1, use_local_extrema=True).add_to(m)

### Dropping Pins

In [33]:
# Drop pins for Top 10 Busiest Stations in Year 2017
yearly_sum_station = df.groupby(["Station"])["Traffic"].sum().sort_values(ascending=False)

top_ten = list(yearly_sum_station.head(10).index)

for i in range(len(top_ten)):

    lat = month_1.loc[month_1["Station"] == top_ten[i]]["lat"]
    lon = month_1.loc[month_1["Station"] == top_ten[i]]["lon"]
    
    popup = folium.Popup('{}'.format(top_ten[i]), parse_html=True) 
    folium.Marker([lat,lon],popup=popup).add_to(m)

m

### Save Interactive HeatMap in HTML

In [16]:
# m.save('map.html')